# BERT의 분류모델 + 코사인 유사도

정리된 csv파일로 처리함


In [4]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F
import numpy as np


In [5]:
df = pd.read_csv('file/df_health_label.csv')

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()

df['category02'].unique(), df['label'].unique()

(array(['예약', '약국', '증상', '수납', '진료접수', '증빙서류', '위치'], dtype=object),
 array([0, 1, 2, 3, 4, 5, 6], dtype=int64))

In [8]:
# 모델 구조를 정의한 코드
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=7)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 가중치를 CPU 또는 GPU로 로드
model.load_state_dict(torch.load('model/health_model_weights_v1.pth', map_location=device))

# 모델을 GPU 또는 CPU로 이동
model = model.to(device)

# 신뢰할 수 있는 데이터 파일을 로드하고 GPU 또는 CPU로 이동
question_embeddings = torch.load('embeddings/train_question_embeddings_v1.pth', map_location=device)

tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")

model.eval()

# 만약 question_embeddings가 numpy 배열로 되어 있다면, 이를 torch.Tensor로 변환
if isinstance(question_embeddings, np.ndarray):
    question_embeddings = [torch.tensor(embedding).to(device) for embedding in question_embeddings]  # numpy 배열을 텐서로 변환


def get_embedding(input_question, tokenizer, model, device):
    # 입력 문장을 토크나이즈하고 GPU/CPU로 이동
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
    
    return cls_embedding.squeeze().to(device)  # 텐서를 GPU로 반환

# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model, device):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model, device)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # question_embedding이 numpy 배열인 경우 텐서로 변환하고 GPU로 이동
        if isinstance(question_embedding, np.ndarray):
            question_embedding = torch.tensor(question_embedding).to(device)
        
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), question_embedding.unsqueeze(0)).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers, df, device):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)  # 이미 model이 device로 이동되어 있음
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i].to(device) for i in filtered_indices]  # GPU로 이동
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model, device)
    
    return best_answer, cosine_similarity, predicted_label


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Temp\ipykernel_14188\3600000252.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. W

In [9]:
# 예시 질문
input_question = "너무 목이 마른데요, 혹시 정수기 같은 건 없나요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(
    input_question, tokenizer, model, question_embeddings, answers, df, device
)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 6
최고 유사도 답변: 정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.
코사인 유사도: 1.0


In [11]:
# test_questions 데이터프레임 읽기
test_questions = pd.read_csv('file/test_questions_with_labels.csv')

# 모든 질문을 리스트로 변환하여 input_questions에 저장
input_questions = test_questions['question'].tolist()
input_labels = test_questions['label'].tolist()  # 라벨 리스트 추가

# 결과를 저장할 리스트 초기화
results = []

# 각 질문에 대해 챗봇 응답 호출
for input_question, input_label in zip(input_questions, input_labels):
    # chatbot_response 함수 호출, 반환 값이 올바른지 확인
    best_answer, cosine_similarity, predicted_label = chatbot_response(
        input_question, tokenizer, model, question_embeddings, answers, df, device
    )
    
    # 결과를 리스트에 추가
    results.append({
        "question": input_question,
        "label": input_label,  # 원래 라벨 추가
        "predicted_label": predicted_label,  # 예측된 라벨 추가
        "best_answer": best_answer,
        "cosine_similarity": cosine_similarity
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력
results_df[:30]


,question,label,predicted_label,best_answer,cosine_similarity
0,화상연고도 처방이 되나요?,1,6,사람이 없는 주사실로 임시 대기를 해주시길 부탁드립니다.,0.770845
1,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,1,4,앞선 예약이 없어서 가능합니다. 예약 시간 한 시간 미뤄 드리겠습니다.,0.983883
2,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,1,4,앞선 예약이 없어서 가능합니다. 예약시간 한 시간 미뤄 드리겠습니다.,0.990516
3,"피가 계속 나는데, 저 죽는 걸까요?",1,2,2층에 있는 검사실에서 알레르기 검사를 받을 수 있습니다.,0.955061
4,접수 재등록해도 되나요?,1,4,예약했어도 접수를 해 주셔야 합니다. 앞에 있는 접수증을 작성해 주시길 부탁드립니다.,0.995472
5,접수 시간이 지났는데 혹시 대기순번이 넘어갔나요?,1,4,네 접수처 직원에게 제출해 주시면 됩니다.,0.996580
6,접수가 왜이렇게 오래 걸려요?,1,4,고객님 기다리게 해서 죄송합니다. 빠른 접수 도와드리겠습니다. 접수할 항목을 말씀해...,0.987359
7,"다음에 들어가는 순번인데, 잠깐 화장실 다녀와도 되나요?",1,6,다음 진료실은 2층입니다. 현재 대기시간 5분 예상됩니다.,0.958992
8,접수하려고요.,1,4,네 처음 오셨으면 종이에 적어 주시고 재진이시면 성함만 불러주세요.,0.986467
9,진료 접수 어떻게 하죠?,1,4,바로 도와드리겠습니다. 옆쪽 통로를 통해 접수창구로 가시면 됩니다.,0.995087


In [26]:
# 정확도 계산
correct_predictions = (results_df['label'] == results_df['predicted_label']).sum()  # 올바른 예측 수
total_predictions = results_df.shape[0]  # 총 예측 수
accuracy = correct_predictions / total_predictions  # 정확도 계산

# 정확도 출력
print(f'정확도: {accuracy * 100:.2f}%')

정확도: 90.14%


In [6]:
results_df.to_csv('result csv/new_chatbot_responses_trained.csv', index=False)